In [1]:
import geopandas as gpd
import pandas as pd
import networkx as nx
import osmnx as ox
import folium
from math import sqrt
from shapely import wkt
import json
from shapely.geometry import Polygon, LineString
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', None)

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Помощь

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Определение функции для поиска и удаления дубликатов
def find_and_remove_duplicates(gdf, name_column, geometry_column, population_column):
    tolerance_km = 40
    tolerance_deg = tolerance_km / 111  # Преобразование километров в градусы (приблизительно)
    
    # Создание пространственного индекса
    gdf = gdf.reset_index(drop=True)
    gdf['duplicate'] = False
    
    spatial_index = gdf.sindex
    
    for idx, row in gdf.iterrows():
        if gdf.at[idx, 'duplicate']:
            continue
        
        # Поиск ближайших точек в пределах допустимого расстояния
        possible_matches_index = list(spatial_index.intersection(row[geometry_column].buffer(tolerance_deg).bounds))
        possible_matches = gdf.iloc[possible_matches_index]
        exact_matches = possible_matches[possible_matches[name_column] == row[name_column]]
        exact_matches = exact_matches[exact_matches[geometry_column].distance(row[geometry_column]) < tolerance_deg]
        
        # Проверка разницы в населении
        exact_matches = exact_matches[exact_matches[population_column].apply(lambda x: x <= row[population_column] * 1.5 and x >= row[population_column] / 1.5)]
        
        if len(exact_matches) > 1:
            duplicate_indices = exact_matches.index
            # Помечаем все, кроме одного, как дубликаты
            gdf.loc[duplicate_indices[1:], 'duplicate'] = True

    return gdf

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
import matplotlib.colors as mcolors
import numpy as np

gdf = gpd.GeoDataFrame(towns)


# Функция для создания квадратного полигона вокруг точки
def create_square(center, size):
    half_size = size / 2.0
    return Polygon([
        (center.x - half_size, center.y - half_size),
        (center.x + half_size, center.y - half_size),
        (center.x + half_size, center.y + half_size),
        (center.x - half_size, center.y + half_size)
    ])

# Определяем размеры квадратов на основе населения
def size_from_population(population, level):
    if level in ["Малое сельское поселение", "Большое сельское поселение", "Крупное сельское поселение", "Малый город или поселок"]:
        return 0.001 * (np.log(population + 1) + 8)  # Логарифмическая шкала для малых населенных пунктов
    return 0.0001 * (population ** 0.5)  # Линейная шкала для крупных населенных пунктов

# Создание квадратов
def convert_points_to_squares(gdf):
    new_geometries = []
    for idx, row in gdf.iterrows():
        if isinstance(row['geometry'], Point):
            size = size_from_population(row['population'], row['level'])
            square = create_square(row['geometry'], size)
            new_geometries.append(square)
        else:
            new_geometries.append(row['geometry'])
    gdf['geometry'] = new_geometries
    return gdf

gdf = convert_points_to_squares(gdf)

# Функция для генерации цветовой карты
def get_color_map(levels):
    base_colors = ['#FFC100', '#FF6500', '#C40C0C', '#6C0345']
    cmap = mcolors.LinearSegmentedColormap.from_list("heatmap", base_colors, N=len(levels))
    norm = mcolors.Normalize(vmin=0, vmax=len(levels) - 1)
    colors = [mcolors.to_hex(cmap(norm(i))) for i in range(len(levels))]
    return colors

# Получение цветов для уровней
levels = list(gdf['level'].unique())
levels.sort(key=lambda x: ["Малое сельское поселение", "Среднее сельское поселение", "Большое сельское поселение", "Крупное сельское поселение", "Малый город или поселок", "Средний город", "Большой город", "Крупный город", "Крупнейший город", "Сверхкрупный город"].index(x))
level_colors = dict(zip(levels, get_color_map(levels)))

# Визуализация на карте
m = folium.Map(location=[43.58548, 39.72311], zoom_start=10, tiles='CartoDB Positron')

for _, row in gdf.iterrows():
    color = level_colors[row['level']]
    popup_text = f"{row['name']}<br>Размер: {row['level']}<br>Население: {row['population']}"
    
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': color,
            'weight': 2,
            'fillOpacity': 0.6
        },
        tooltip=popup_text
    ).add_to(m)

# Создание легенды
legend_html = '<div style="position: fixed; bottom: 50px; left: 50px; width: 300px; height: 350px; border:2px solid grey; z-index:9999; font-size:14px; background-color:white; padding: 10px;"><b>Легенда</b><br>'
for level, color in level_colors.items():
    legend_html += f'&nbsp; {level} &nbsp; <i class="fa fa-square" style="font-size:20px;color:{color}"></i><br>'
legend_html += '</div>'

# Добавление легенды на карту
m.get_root().html.add_child(folium.Element(legend_html))

# Сохранение карты в HTML-файл
m.save('output.html')

## Тюмень

In [4]:
towns = gpd.read_file('data_9/4882/towns.geojson')
okrugs = gpd.read_file('data_9/4882/region.geojson')
rayons = gpd.read_file('data_9/4882\districts.geojson')
adj_mx = pd.read_pickle("data_9/4882/adj_mx.pickle")


In [5]:
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

,geometry,name,population,level
0,POINT (70.15830 55.20878),Красивое,192,Малое сельское поселение
1,POINT (69.94983 55.26997),Александровка,629,Большое сельское поселение
2,POINT (70.08474 55.26131),Михайловка,184,Малое сельское поселение
3,POINT (70.09869 55.43390),Покровка,240,Большое сельское поселение
4,POINT (70.15640 55.34947),Таволжан,181,Малое сельское поселение
...,...,...,...,...
1206,POINT (65.27978 57.44325),Новоказанка,22,Малое сельское поселение
1207,POINT (65.37807 57.47281),Новопокровка,361,Большое сельское поселение
1208,POINT (64.97423 57.53028),Ахманы,104,Малое сельское поселение
1209,POINT (65.36232 57.36702),Средние Тарманы,489,Большое сельское поселение


In [6]:
okrugs = gpd.read_file('data_9/4882/region.geojson')
rayons = gpd.read_file('data_9/4882\districts.geojson')
adj_mx = pd.read_pickle("data_9/4882/adj_mx.pickle")

In [7]:
from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=okrugs, 
  districts=rayons, 
  accessibility_matrix=adj_mx,
  territories=None
)

In [8]:
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame

,geometry,name,level,population
0,POINT (70.15830 55.20878),Красивое,Малое сельское поселение,192.0
1,POINT (69.94983 55.26997),Александровка,Большое сельское поселение,629.0
2,POINT (70.08474 55.26131),Михайловка,Малое сельское поселение,184.0
3,POINT (70.09869 55.43390),Покровка,Большое сельское поселение,240.0
4,POINT (70.15640 55.34947),Таволжан,Малое сельское поселение,181.0
...,...,...,...,...
2416,"LINESTRING (65.18848 56.95751, 65.07397 56.97362)",NaN,Малое сельское поселение,NaN
2417,"LINESTRING (65.30903 57.53569, 65.16233 57.50669)",NaN,Малое сельское поселение,NaN
2418,"LINESTRING (65.16233 57.50669, 65.16725 57.53747)",NaN,Малое сельское поселение,NaN
2419,"LINESTRING (65.16233 57.50669, 65.27978 57.44325)",NaN,Малое сельское поселение,NaN


In [9]:
# gdf_frame.to_file("gdf_frame_tumen.geojson", driver="GeoJSON")
# frame_method.get_graph_html(G, 'final_graph_with_legend.html')

## Московская область

In [12]:
towns = gpd.read_parquet('data_9/1111/towns.parquet')
region = gpd.read_file('data_9/1111/region.geojson')
districts = gpd.read_file('data_9/1111\districts.geojson')
# adj_mx = pd.read_pickle("data_9/1111/adj_mx.pickle")
graph = nx.read_graphml('data_9/1111/graph.graphml')

In [91]:
# # Применение функции для поиска и удаления дубликатов
# gdf_with_duplicates = find_and_remove_duplicates(towns, 'name', 'geometry', 'population')

# # Удаление дубликатов из исходного GeoDataFrame
# towns = gdf_with_duplicates[~gdf_with_duplicates['duplicate']].drop(columns=['duplicate'])

# towns.reset_index(drop=True, inplace=True)
# towns.to_parquet('towns.parquet', engine='pyarrow')
# towns

C:\Users\max\AppData\Local\Temp\ipykernel_35304\761477160.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  exact_matches = exact_matches[exact_matches[geometry_column].distance(row[geometry_column]) < tolerance_deg]


,name,population,geometry
0,Серебряные Пруды,9358,POINT (38.72237 54.46798)
1,Узуново,3417,POINT (38.61833 54.54061)
2,Благодать,52,POINT (38.62932 54.47175)
3,Ступино,66463,POINT (38.07726 54.88660)
4,Кашира,39929,POINT (38.15051 54.83745)
...,...,...,...
2884,Шелгуново,9,POINT (35.60816 56.37178)
2885,Мазлово,2,POINT (35.68766 56.36313)
2886,Себудово,2,POINT (35.52194 56.37250)
2887,Введенское,694,POINT (35.63585 56.37377)


In [13]:
from popframe.preprocessing.adjacency_calculator import AdjacencyCalculator
blocks = towns.copy()
blocks.geometry = blocks.geometry.buffer(0.01)
ac = AdjacencyCalculator(blocks=blocks, graph=graph)
adj_mx = ac.get_dataframe()

C:\Users\max\AppData\Local\Temp\ipykernel_48928\2945463897.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.01)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [14]:
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

,geometry,name,population,level
0,POINT (38.72237 54.46798),Серебряные Пруды,9358,Малый город
1,POINT (38.61833 54.54061),Узуново,3417,Крупное сельское поселение
2,POINT (38.62932 54.47175),Благодать,52,Малое сельское поселение
3,POINT (38.07726 54.88660),Ступино,66463,Средний город
4,POINT (38.15051 54.83745),Кашира,39929,Малый город
...,...,...,...,...
2873,POINT (35.60816 56.37178),Шелгуново,9,Малое сельское поселение
2874,POINT (35.68766 56.36313),Мазлово,2,Малое сельское поселение
2875,POINT (35.52194 56.37250),Себудово,2,Малое сельское поселение
2876,POINT (35.63585 56.37377),Введенское,694,Большое сельское поселение


In [15]:
from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=region, 
  districts=districts, 
  accessibility_matrix=adj_mx,
  territories=None
)

In [16]:
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame

,geometry,name,level,population
0,POINT (38.72237 54.46798),Серебряные Пруды,Малый город,9358.0
1,POINT (38.61833 54.54061),Узуново,Крупное сельское поселение,3417.0
2,POINT (38.62932 54.47175),Благодать,Малое сельское поселение,52.0
3,POINT (38.07726 54.88660),Ступино,Средний город,66463.0
4,POINT (38.15051 54.83745),Кашира,Малый город,39929.0
...,...,...,...,...
5741,"LINESTRING (35.50998 56.36097, 35.63585 56.37377)",NaN,Малое сельское поселение,NaN
5742,"LINESTRING (35.62201 56.37238, 35.63585 56.37377)",NaN,Малое сельское поселение,NaN
5743,"LINESTRING (35.60816 56.37178, 35.63585 56.37377)",NaN,Малое сельское поселение,NaN
5744,"LINESTRING (35.68766 56.36313, 35.63585 56.37377)",NaN,Малое сельское поселение,NaN


In [17]:
# gdf_frame.to_file("gdf_frame_MO.geojson", driver="GeoJSON")
# frame_method.get_graph_html(G, 'final_graph_MO.html')

## Краснодар

In [3]:
towns = gpd.read_parquet('data_9/2222/towns.parquet')
# towns = gpd.read_file('data_9/2222/towns.geojson')
region = gpd.read_file('data_9/2222/region.geojson')
districts = gpd.read_file('data_9/2222\districts.geojson')
# adj_mx = pd.read_pickle("data_9/1111/adj_mx.pickle")
graph = nx.read_graphml('data_9/2222/graph.graphml')

In [4]:
towns

,name,population,geometry
0,Сочи,446599,POINT (39.72311 43.58548)
1,Дагомыс,17841,POINT (39.65444 43.65900)
2,Солохаул,214,POINT (39.67738 43.80080)
3,Пластунка,2270,POINT (39.76183 43.66878)
4,Нижняя Шиловка,5432,POINT (40.02332 43.45000)
...,...,...,...
1603,Солёный,486,POINT (37.03120 45.26675)
1604,Артющенко,90,POINT (36.81234 45.13542)
1605,Виноградный,1988,POINT (36.89917 45.19328)
1606,Пересыпь,776,POINT (37.13355 45.34775)


In [5]:
# Применение функции для поиска и удаления дубликатов
gdf_with_duplicates = find_and_remove_duplicates(towns, 'name', 'geometry', 'population')

# Удаление дубликатов из исходного GeoDataFrame
towns = gdf_with_duplicates[~gdf_with_duplicates['duplicate']].drop(columns=['duplicate'])

towns.reset_index(drop=True, inplace=True)
# towns.to_parquet('towns.parquet', engine='pyarrow')
# towns

C:\Users\max\AppData\Local\Temp\ipykernel_39608\2491489582.py:24: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  exact_matches = exact_matches[exact_matches[geometry_column].distance(row[geometry_column]) < tolerance_deg]


,name,population,geometry
0,Сочи,446599,POINT (39.72311 43.58548)
1,Дагомыс,17841,POINT (39.65444 43.65900)
2,Солохаул,214,POINT (39.67738 43.80080)
3,Пластунка,2270,POINT (39.76183 43.66878)
4,Нижняя Шиловка,5432,POINT (40.02332 43.45000)
...,...,...,...
1597,Солёный,486,POINT (37.03120 45.26675)
1598,Артющенко,90,POINT (36.81234 45.13542)
1599,Виноградный,1988,POINT (36.89917 45.19328)
1600,Пересыпь,776,POINT (37.13355 45.34775)


In [6]:
from popframe.preprocessing.adjacency_calculator import AdjacencyCalculator
blocks = towns.copy()
blocks.geometry = blocks.geometry.buffer(0.01)
ac = AdjacencyCalculator(blocks=blocks, graph=graph)
adj_mx = ac.get_dataframe()

C:\Users\max\AppData\Local\Temp\ipykernel_39608\2945463897.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks.geometry = blocks.geometry.buffer(0.01)
c:\Code\PopFrame\.venv\lib\site-packages\geopandas\array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [7]:
from popframe.preprocessing.level_filler import LevelFiller
level_filler = LevelFiller(towns=towns)
towns = level_filler.fill_levels()
towns

,geometry,name,population,level
0,POINT (39.72311 43.58548),Сочи,446599,Крупный город
1,POINT (39.65444 43.65900),Дагомыс,17841,Малый город
2,POINT (39.67738 43.80080),Солохаул,214,Большое сельское поселение
3,POINT (39.76183 43.66878),Пластунка,2270,Крупное сельское поселение
4,POINT (40.02332 43.45000),Нижняя Шиловка,5432,Малый город
...,...,...,...,...
1597,POINT (37.03120 45.26675),Солёный,486,Большое сельское поселение
1598,POINT (36.81234 45.13542),Артющенко,90,Малое сельское поселение
1599,POINT (36.89917 45.19328),Виноградный,1988,Крупное сельское поселение
1600,POINT (37.13355 45.34775),Пересыпь,776,Большое сельское поселение


In [8]:
from popframe.models.region import Region

region = Region(
  towns=towns,
  settlements=region, 
  districts=districts, 
  accessibility_matrix=adj_mx,
  territories=None
)

In [9]:
from popframe.method.popuation_frame import PopFrame
# Создание экземпляра GraphMethod с данными
frame_method = PopFrame(region=region)

# Получение графа
G = frame_method.build_network()
gdf_frame = frame_method.save_graph_to_geojson(G, None)
gdf_frame

,geometry,name,level,population
0,POINT (39.72311 43.58548),Сочи,Крупный город,446599.0
1,POINT (39.65444 43.65900),Дагомыс,Малый город,17841.0
2,POINT (39.67738 43.80080),Солохаул,Большое сельское поселение,214.0
3,POINT (39.76183 43.66878),Пластунка,Крупное сельское поселение,2270.0
4,POINT (40.02332 43.45000),Нижняя Шиловка,Малый город,5432.0
...,...,...,...,...
3198,"LINESTRING (36.93308 45.36573, 36.94821 45.33243)",NaN,Большое сельское поселение,NaN
3199,"LINESTRING (36.98520 45.33221, 37.03120 45.26675)",NaN,Большое сельское поселение,NaN
3200,"LINESTRING (37.10490 45.32134, 37.13355 45.34775)",NaN,Большое сельское поселение,NaN
3201,"LINESTRING (36.81476 45.35401, 36.85218 45.31984)",NaN,Большое сельское поселение,NaN


In [10]:
nodes = list(G.nodes(data=True))
isolated_nodes = [node for node in nodes if G.degree(node[0]) == 0]
isolated_nodes

[]

In [11]:
# gdf_frame.to_file("gdf_frame_Krasnodar.geojson", driver="GeoJSON")
# frame_method.get_graph_html(G, 'final_graph_Krasnodar.html')

## Омская область